In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import array
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import train_test_split 
from sklearn import linear_model, preprocessing 
import warnings
import statsmodels.api as sm
!pip install GEKKO
!pip install ortools

from gekko import GEKKO
import math

In [3]:
projections=pd.read_csv("DFF_NBA_cheatsheet_2021-12-20.csv")

In [4]:
projections

,first_name,last_name,position,position_alt,injury_status,game_date,slate,team,opp,spread,...,implied_team_score,salary,L5_dvp_rank,L5_ppg_floor,L5_ppg_avg,L5_ppg_max,ppg_projection,value_projection,ppg_actual,value_actual
0,Stephen,Curry,PG,NaN,NaN,2021-12-20,NaN,GSW,SAC,-12.5,...,117.0,11800,27,30.5,38.3,45.5,54.5,4.62,NaN,NaN
1,Joel,Embiid,C,NaN,Q,2021-12-20,NaN,PHI,BOS,3.0,...,105.3,11600,18,41.3,47.7,53.8,51.4,4.43,NaN,NaN
2,Jayson,Tatum,SF,PF,Q,2021-12-20,NaN,BOS,PHI,-3.0,...,108.3,10500,30,37.8,47.7,62.3,48.1,4.58,NaN,NaN
3,Dejounte,Murray,PG,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,9800,13,21.8,41.4,54.8,44.5,4.54,NaN,NaN
4,Rudy,Gobert,C,NaN,NaN,2021-12-20,NaN,UTA,CHA,-11.5,...,124.0,9200,8,37.3,45.4,50.3,44.3,4.81,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Marvin,Bagley III,PF,NaN,O,2021-12-20,NaN,SAC,GSW,12.5,...,104.5,4600,20,17.8,26.3,34.5,0.0,0.00,NaN,NaN
168,Cody,Martin,SG,SF,O,2021-12-20,NaN,CHA,UTA,11.5,...,112.5,5700,4,17.0,32.0,45.5,0.0,0.00,NaN,NaN
169,Troy,Brown Jr.,SF,PF,O,2021-12-20,NaN,CHI,HOU,-7.0,...,114.8,3600,13,9.5,17.0,23.0,0.0,0.00,NaN,NaN
170,Andrew,Wiggins,SG,SF,O,2021-12-20,NaN,GSW,SAC,-12.5,...,117.0,6800,12,25.0,33.0,44.0,0.0,0.00,NaN,NaN


In [10]:
SA_projections=projections.loc[projections['team'] == 'SA']

In [11]:
LAC_projections=projections.loc[projections['team'] == 'LAC']

In [13]:
SA_projections

,first_name,last_name,position,position_alt,injury_status,game_date,slate,team,opp,spread,...,implied_team_score,salary,L5_dvp_rank,L5_ppg_floor,L5_ppg_avg,L5_ppg_max,ppg_projection,value_projection,ppg_actual,value_actual
3,Dejounte,Murray,PG,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,9800,13,21.8,41.4,54.8,44.5,4.54,NaN,NaN
24,Jakob,Poeltl,C,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,6200,4,15.0,26.9,49.0,32.2,5.19,NaN,NaN
27,Derrick,White,PG,SG,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,6500,13,12.8,31.6,57.0,31.0,4.77,NaN,NaN
41,Keldon,Johnson,PF,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,5400,9,22.0,30.3,38.5,26.5,4.91,NaN,NaN
62,Lonnie,Walker IV,SG,SF,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,4200,23,14.0,26.4,37.0,20.5,4.88,NaN,NaN
66,Devin,Vassell,SF,PF,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,4100,21,14.3,20.2,28.3,19.8,4.82,NaN,NaN
90,Bryn,Forbes,SG,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,3400,23,0.0,15.2,29.3,15.5,4.56,NaN,NaN
96,Tre,Jones,PG,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,3000,13,4.0,13.6,30.8,13.8,4.58,NaN,NaN
99,Jock,Landale,C,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,3100,4,0.0,6.2,12.0,13.3,4.27,NaN,NaN
113,Keita,Bates-Diop,SF,PF,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,3000,21,-0.5,14.2,24.8,10.0,3.33,NaN,NaN


In [14]:
LAC_projections

,first_name,last_name,position,position_alt,injury_status,game_date,slate,team,opp,spread,...,implied_team_score,salary,L5_dvp_rank,L5_ppg_floor,L5_ppg_avg,L5_ppg_max,ppg_projection,value_projection,ppg_actual,value_actual
5,Paul,George,SG,SF,Q,2021-12-20,NaN,LAC,SA,-4.5,...,110.8,10800,13,35.3,41.3,45.0,43.5,4.03,NaN,NaN
25,Ivica,Zubac,C,NaN,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,110.8,4900,29,15.5,22.1,33.5,31.5,6.43,NaN,NaN
26,Reggie,Jackson,PG,NaN,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,110.8,7600,15,25.8,34.0,42.5,31.5,4.14,NaN,NaN
47,Luke,Kennard,SG,NaN,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,110.8,6000,13,17.5,31.4,45.8,24.6,4.10,NaN,NaN
51,Terance,Mann,PG,SF,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,110.8,5400,15,11.3,27.5,33.8,23.3,4.31,NaN,NaN
55,Eric,Bledsoe,PG,NaN,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,110.8,4700,15,16.3,24.8,42.8,22.3,4.73,NaN,NaN
61,Nicolas,Batum,PF,C,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,110.8,4500,5,4.3,13.9,21.3,20.8,4.61,NaN,NaN
92,Brandon,Boston Jr.,SG,SF,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,110.8,3300,13,1.5,14.5,39.0,14.8,4.47,NaN,NaN
104,Amir,Coffey,PG,SG,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,110.8,3000,15,1.3,7.0,13.8,13.0,4.33,NaN,NaN
106,Justise,Winslow,PG,SF,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,110.8,3400,15,6.0,14.4,24.3,12.6,3.71,NaN,NaN


In [15]:
result = SA_projections.append(LAC_projections)

In [16]:
result

,first_name,last_name,position,position_alt,injury_status,game_date,slate,team,opp,spread,...,implied_team_score,salary,L5_dvp_rank,L5_ppg_floor,L5_ppg_avg,L5_ppg_max,ppg_projection,value_projection,ppg_actual,value_actual
3,Dejounte,Murray,PG,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,9800,13,21.8,41.4,54.8,44.5,4.54,NaN,NaN
24,Jakob,Poeltl,C,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,6200,4,15.0,26.9,49.0,32.2,5.19,NaN,NaN
27,Derrick,White,PG,SG,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,6500,13,12.8,31.6,57.0,31.0,4.77,NaN,NaN
41,Keldon,Johnson,PF,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,5400,9,22.0,30.3,38.5,26.5,4.91,NaN,NaN
62,Lonnie,Walker IV,SG,SF,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,4200,23,14.0,26.4,37.0,20.5,4.88,NaN,NaN
66,Devin,Vassell,SF,PF,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,4100,21,14.3,20.2,28.3,19.8,4.82,NaN,NaN
90,Bryn,Forbes,SG,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,3400,23,0.0,15.2,29.3,15.5,4.56,NaN,NaN
96,Tre,Jones,PG,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,3000,13,4.0,13.6,30.8,13.8,4.58,NaN,NaN
99,Jock,Landale,C,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,3100,4,0.0,6.2,12.0,13.3,4.27,NaN,NaN
113,Keita,Bates-Diop,SF,PF,NaN,2021-12-20,NaN,SA,LAC,4.5,...,106.3,3000,21,-0.5,14.2,24.8,10.0,3.33,NaN,NaN


In [17]:
result["Name"] = result["first_name"] +' '+ result["last_name"]

In [18]:
result

,first_name,last_name,position,position_alt,injury_status,game_date,slate,team,opp,spread,...,salary,L5_dvp_rank,L5_ppg_floor,L5_ppg_avg,L5_ppg_max,ppg_projection,value_projection,ppg_actual,value_actual,Name
3,Dejounte,Murray,PG,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,9800,13,21.8,41.4,54.8,44.5,4.54,NaN,NaN,Dejounte Murray
24,Jakob,Poeltl,C,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,6200,4,15.0,26.9,49.0,32.2,5.19,NaN,NaN,Jakob Poeltl
27,Derrick,White,PG,SG,NaN,2021-12-20,NaN,SA,LAC,4.5,...,6500,13,12.8,31.6,57.0,31.0,4.77,NaN,NaN,Derrick White
41,Keldon,Johnson,PF,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,5400,9,22.0,30.3,38.5,26.5,4.91,NaN,NaN,Keldon Johnson
62,Lonnie,Walker IV,SG,SF,NaN,2021-12-20,NaN,SA,LAC,4.5,...,4200,23,14.0,26.4,37.0,20.5,4.88,NaN,NaN,Lonnie Walker IV
66,Devin,Vassell,SF,PF,NaN,2021-12-20,NaN,SA,LAC,4.5,...,4100,21,14.3,20.2,28.3,19.8,4.82,NaN,NaN,Devin Vassell
90,Bryn,Forbes,SG,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,3400,23,0.0,15.2,29.3,15.5,4.56,NaN,NaN,Bryn Forbes
96,Tre,Jones,PG,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,3000,13,4.0,13.6,30.8,13.8,4.58,NaN,NaN,Tre Jones
99,Jock,Landale,C,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,3100,4,0.0,6.2,12.0,13.3,4.27,NaN,NaN,Jock Landale
113,Keita,Bates-Diop,SF,PF,NaN,2021-12-20,NaN,SA,LAC,4.5,...,3000,21,-0.5,14.2,24.8,10.0,3.33,NaN,NaN,Keita Bates-Diop


In [19]:
salaries=pd.read_csv("DKSalaries-7.csv")

In [20]:
salaries

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,SG/SF,Paul George (20688335),Paul George,20688335,CPT,18600,SAS@LAC 12/20/2021 10:30PM ET,LAC,46.28
1,PF,Kawhi Leonard (20688336),Kawhi Leonard,20688336,CPT,17400,SAS@LAC 12/20/2021 10:30PM ET,LAC,0.00
2,PG,Dejounte Murray (20688337),Dejounte Murray,20688337,CPT,17100,SAS@LAC 12/20/2021 10:30PM ET,SAS,46.33
3,PG,Reggie Jackson (20688338),Reggie Jackson,20688338,CPT,13800,SAS@LAC 12/20/2021 10:30PM ET,LAC,30.35
4,SG/SF,Paul George (20688377),Paul George,20688377,UTIL,12400,SAS@LAC 12/20/2021 10:30PM ET,LAC,46.28
...,...,...,...,...,...,...,...,...,...
79,SG,Jaylen Morris (20688414),Jaylen Morris,20688414,UTIL,1000,SAS@LAC 12/20/2021 10:30PM ET,SAS,0.00
80,SG/SF,Denzel Mahoney (20688415),Denzel Mahoney,20688415,UTIL,1000,SAS@LAC 12/20/2021 10:30PM ET,SAS,0.00
81,PF/C,Serge Ibaka (20688416),Serge Ibaka,20688416,UTIL,1000,SAS@LAC 12/20/2021 10:30PM ET,LAC,10.91
82,SG,Joshua Primo (20688417),Joshua Primo,20688417,UTIL,1000,SAS@LAC 12/20/2021 10:30PM ET,SAS,5.69


In [21]:
result2 = pd.merge(result,
                 salaries,
                 on='Name', 
                 how='inner')

In [22]:
result2

,first_name,last_name,position,position_alt,injury_status,game_date,slate,team,opp,spread,...,value_actual,Name,Position,Name + ID,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,Dejounte,Murray,PG,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Dejounte Murray,PG,Dejounte Murray (20688337),20688337,CPT,17100,SAS@LAC 12/20/2021 10:30PM ET,SAS,46.33
1,Dejounte,Murray,PG,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Dejounte Murray,PG,Dejounte Murray (20688379),20688379,UTIL,11400,SAS@LAC 12/20/2021 10:30PM ET,SAS,46.33
2,Jakob,Poeltl,C,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Jakob Poeltl,C,Jakob Poeltl (20688340),20688340,CPT,11400,SAS@LAC 12/20/2021 10:30PM ET,SAS,30.62
3,Jakob,Poeltl,C,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Jakob Poeltl,C,Jakob Poeltl (20688382),20688382,UTIL,7600,SAS@LAC 12/20/2021 10:30PM ET,SAS,30.62
4,Derrick,White,PG,SG,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Derrick White,PG/SG,Derrick White (20688339),20688339,CPT,12000,SAS@LAC 12/20/2021 10:30PM ET,SAS,30.39
5,Derrick,White,PG,SG,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Derrick White,PG/SG,Derrick White (20688381),20688381,UTIL,8000,SAS@LAC 12/20/2021 10:30PM ET,SAS,30.39
6,Keldon,Johnson,PF,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Keldon Johnson,PF,Keldon Johnson (20688343),20688343,CPT,10500,SAS@LAC 12/20/2021 10:30PM ET,SAS,28.01
7,Keldon,Johnson,PF,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Keldon Johnson,PF,Keldon Johnson (20688385),20688385,UTIL,7000,SAS@LAC 12/20/2021 10:30PM ET,SAS,28.01
8,Lonnie,Walker IV,SG,SF,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Lonnie Walker IV,SG/SF,Lonnie Walker IV (20688349),20688349,CPT,6900,SAS@LAC 12/20/2021 10:30PM ET,SAS,20.66
9,Lonnie,Walker IV,SG,SF,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Lonnie Walker IV,SG/SF,Lonnie Walker IV (20688391),20688391,UTIL,4600,SAS@LAC 12/20/2021 10:30PM ET,SAS,20.66


In [23]:
result3=result2.loc[result2['Roster Position'] != 'CPT']

In [24]:
result3

,first_name,last_name,position,position_alt,injury_status,game_date,slate,team,opp,spread,...,value_actual,Name,Position,Name + ID,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
1,Dejounte,Murray,PG,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Dejounte Murray,PG,Dejounte Murray (20688379),20688379,UTIL,11400,SAS@LAC 12/20/2021 10:30PM ET,SAS,46.33
3,Jakob,Poeltl,C,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Jakob Poeltl,C,Jakob Poeltl (20688382),20688382,UTIL,7600,SAS@LAC 12/20/2021 10:30PM ET,SAS,30.62
5,Derrick,White,PG,SG,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Derrick White,PG/SG,Derrick White (20688381),20688381,UTIL,8000,SAS@LAC 12/20/2021 10:30PM ET,SAS,30.39
7,Keldon,Johnson,PF,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Keldon Johnson,PF,Keldon Johnson (20688385),20688385,UTIL,7000,SAS@LAC 12/20/2021 10:30PM ET,SAS,28.01
9,Lonnie,Walker IV,SG,SF,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Lonnie Walker IV,SG/SF,Lonnie Walker IV (20688391),20688391,UTIL,4600,SAS@LAC 12/20/2021 10:30PM ET,SAS,20.66
11,Devin,Vassell,SF,PF,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Devin Vassell,SF/PF,Devin Vassell (20688392),20688392,UTIL,4400,SAS@LAC 12/20/2021 10:30PM ET,SAS,21.99
13,Bryn,Forbes,SG,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Bryn Forbes,SG,Bryn Forbes (20688395),20688395,UTIL,3000,SAS@LAC 12/20/2021 10:30PM ET,SAS,11.59
15,Tre,Jones,PG,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Tre Jones,PG,Tre Jones (20688401),20688401,UTIL,1000,SAS@LAC 12/20/2021 10:30PM ET,SAS,9.06
17,Jock,Landale,C,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Jock Landale,C,Jock Landale (20688399),20688399,UTIL,2000,SAS@LAC 12/20/2021 10:30PM ET,SAS,4.46
19,Keita,Bates-Diop,SF,PF,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Keita Bates-Diop,SF/PF,Keita Bates-Diop (20688406),20688406,UTIL,1000,SAS@LAC 12/20/2021 10:30PM ET,SAS,10.38


In [25]:
result3 = result3.sort_values(by='Salary', ascending=False)

In [26]:
result3

,first_name,last_name,position,position_alt,injury_status,game_date,slate,team,opp,spread,...,value_actual,Name,Position,Name + ID,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
29,Paul,George,SG,SF,Q,2021-12-20,NaN,LAC,SA,-4.5,...,NaN,Paul George,SG/SF,Paul George (20688377),20688377,UTIL,12400,SAS@LAC 12/20/2021 10:30PM ET,LAC,46.28
51,Kawhi,Leonard,PF,NaN,O,2021-12-20,NaN,LAC,SA,-4.5,...,NaN,Kawhi Leonard,PF,Kawhi Leonard (20688378),20688378,UTIL,11600,SAS@LAC 12/20/2021 10:30PM ET,LAC,0.00
1,Dejounte,Murray,PG,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Dejounte Murray,PG,Dejounte Murray (20688379),20688379,UTIL,11400,SAS@LAC 12/20/2021 10:30PM ET,SAS,46.33
33,Reggie,Jackson,PG,NaN,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,NaN,Reggie Jackson,PG,Reggie Jackson (20688380),20688380,UTIL,9200,SAS@LAC 12/20/2021 10:30PM ET,LAC,30.35
5,Derrick,White,PG,SG,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Derrick White,PG/SG,Derrick White (20688381),20688381,UTIL,8000,SAS@LAC 12/20/2021 10:30PM ET,SAS,30.39
3,Jakob,Poeltl,C,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Jakob Poeltl,C,Jakob Poeltl (20688382),20688382,UTIL,7600,SAS@LAC 12/20/2021 10:30PM ET,SAS,30.62
35,Luke,Kennard,SG,NaN,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,NaN,Luke Kennard,SG,Luke Kennard (20688383),20688383,UTIL,7400,SAS@LAC 12/20/2021 10:30PM ET,LAC,22.07
53,Marcus,Morris Sr.,SF,PF,O,2021-12-20,NaN,LAC,SA,-4.5,...,NaN,Marcus Morris Sr.,SF/PF,Marcus Morris Sr. (20688384),20688384,UTIL,7200,SAS@LAC 12/20/2021 10:30PM ET,LAC,24.73
7,Keldon,Johnson,PF,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,NaN,Keldon Johnson,PF,Keldon Johnson (20688385),20688385,UTIL,7000,SAS@LAC 12/20/2021 10:30PM ET,SAS,28.01
37,Terance,Mann,PG,SF,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,NaN,Terance Mann,PG/SF,Terance Mann (20688386),20688386,UTIL,6800,SAS@LAC 12/20/2021 10:30PM ET,LAC,21.95


In [27]:
historic=pd.read_csv("Point_Cost_ratio_and_STD.csv")

In [28]:
historic

,PlayerName,Average,StandardDeviation,Average_STD_Ratio
0,Russell Westbrook,56.608037,12.606392,4.490423
1,Ty Jerome,55.252937,21.725725,2.543203
2,Nikola Jokić,53.883490,11.789333,4.570529
3,Kevin Porter Jr.,53.553873,33.742134,1.587151
4,Kelly Olynyk,53.444172,21.305469,2.508472
...,...,...,...,...
413,Udoka Azubuike,12.083333,11.564176,1.044894
414,Tyrell Terry,10.959433,12.558782,0.872651
415,Tyler Bey,10.547327,7.921265,1.331521
416,Nick Richards,9.237231,6.649471,1.389168


In [31]:
historic["Name"] = historic["PlayerName"]

In [35]:
historic['Name'] = historic['Name'].str.rstrip()

In [36]:
historic

,PlayerName,Average,StandardDeviation,Average_STD_Ratio,Name
0,Russell Westbrook,56.608037,12.606392,4.490423,Russell Westbrook
1,Ty Jerome,55.252937,21.725725,2.543203,Ty Jerome
2,Nikola Jokić,53.883490,11.789333,4.570529,Nikola Jokić
3,Kevin Porter Jr.,53.553873,33.742134,1.587151,Kevin Porter Jr.
4,Kelly Olynyk,53.444172,21.305469,2.508472,Kelly Olynyk
...,...,...,...,...,...
413,Udoka Azubuike,12.083333,11.564176,1.044894,Udoka Azubuike
414,Tyrell Terry,10.959433,12.558782,0.872651,Tyrell Terry
415,Tyler Bey,10.547327,7.921265,1.331521,Tyler Bey
416,Nick Richards,9.237231,6.649471,1.389168,Nick Richards


In [37]:
result4 = pd.merge(result3,
                 historic,
                 on='Name', 
                 how='inner')

In [38]:
result4

,first_name,last_name,position,position_alt,injury_status,game_date,slate,team,opp,spread,...,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,PlayerName,Average,StandardDeviation,Average_STD_Ratio
0,Paul,George,SG,SF,Q,2021-12-20,NaN,LAC,SA,-4.5,...,20688377,UTIL,12400,SAS@LAC 12/20/2021 10:30PM ET,LAC,46.28,Paul George,49.159715,12.474737,3.940742
1,Kawhi,Leonard,PF,NaN,O,2021-12-20,NaN,LAC,SA,-4.5,...,20688378,UTIL,11600,SAS@LAC 12/20/2021 10:30PM ET,LAC,0.00,Kawhi Leonard,49.105433,9.788836,5.016473
2,Dejounte,Murray,PG,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,20688379,UTIL,11400,SAS@LAC 12/20/2021 10:30PM ET,SAS,46.33,Dejounte Murray,50.614523,16.606912,3.047799
3,Reggie,Jackson,PG,NaN,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,20688380,UTIL,9200,SAS@LAC 12/20/2021 10:30PM ET,LAC,30.35,Reggie Jackson,44.393001,23.240824,1.910130
4,Derrick,White,PG,SG,NaN,2021-12-20,NaN,SA,LAC,4.5,...,20688381,UTIL,8000,SAS@LAC 12/20/2021 10:30PM ET,SAS,30.39,Derrick White,48.657340,16.627761,2.926271
5,Jakob,Poeltl,C,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,20688382,UTIL,7600,SAS@LAC 12/20/2021 10:30PM ET,SAS,30.62,Jakob Poeltl,48.995904,19.633243,2.495558
6,Luke,Kennard,SG,NaN,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,20688383,UTIL,7400,SAS@LAC 12/20/2021 10:30PM ET,LAC,22.07,Luke Kennard,40.398458,28.104520,1.437436
7,Keldon,Johnson,PF,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,20688385,UTIL,7000,SAS@LAC 12/20/2021 10:30PM ET,SAS,28.01,Keldon Johnson,45.990018,20.052901,2.293435
8,Terance,Mann,PG,SF,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,20688386,UTIL,6800,SAS@LAC 12/20/2021 10:30PM ET,LAC,21.95,Terance Mann,44.048070,29.204982,1.508238
9,Ivica,Zubac,C,NaN,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,20688387,UTIL,5800,SAS@LAC 12/20/2021 10:30PM ET,LAC,24.18,Ivica Zubac,46.213078,18.835737,2.453479


In [39]:
result4=result4.loc[result4['injury_status'] != 'O']

In [40]:
result4

,first_name,last_name,position,position_alt,injury_status,game_date,slate,team,opp,spread,...,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,PlayerName,Average,StandardDeviation,Average_STD_Ratio
0,Paul,George,SG,SF,Q,2021-12-20,NaN,LAC,SA,-4.5,...,20688377,UTIL,12400,SAS@LAC 12/20/2021 10:30PM ET,LAC,46.28,Paul George,49.159715,12.474737,3.940742
2,Dejounte,Murray,PG,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,20688379,UTIL,11400,SAS@LAC 12/20/2021 10:30PM ET,SAS,46.33,Dejounte Murray,50.614523,16.606912,3.047799
3,Reggie,Jackson,PG,NaN,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,20688380,UTIL,9200,SAS@LAC 12/20/2021 10:30PM ET,LAC,30.35,Reggie Jackson,44.393001,23.240824,1.910130
4,Derrick,White,PG,SG,NaN,2021-12-20,NaN,SA,LAC,4.5,...,20688381,UTIL,8000,SAS@LAC 12/20/2021 10:30PM ET,SAS,30.39,Derrick White,48.657340,16.627761,2.926271
5,Jakob,Poeltl,C,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,20688382,UTIL,7600,SAS@LAC 12/20/2021 10:30PM ET,SAS,30.62,Jakob Poeltl,48.995904,19.633243,2.495558
6,Luke,Kennard,SG,NaN,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,20688383,UTIL,7400,SAS@LAC 12/20/2021 10:30PM ET,LAC,22.07,Luke Kennard,40.398458,28.104520,1.437436
7,Keldon,Johnson,PF,NaN,NaN,2021-12-20,NaN,SA,LAC,4.5,...,20688385,UTIL,7000,SAS@LAC 12/20/2021 10:30PM ET,SAS,28.01,Keldon Johnson,45.990018,20.052901,2.293435
8,Terance,Mann,PG,SF,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,20688386,UTIL,6800,SAS@LAC 12/20/2021 10:30PM ET,LAC,21.95,Terance Mann,44.048070,29.204982,1.508238
9,Ivica,Zubac,C,NaN,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,20688387,UTIL,5800,SAS@LAC 12/20/2021 10:30PM ET,LAC,24.18,Ivica Zubac,46.213078,18.835737,2.453479
10,Eric,Bledsoe,PG,NaN,NaN,2021-12-20,NaN,LAC,SA,-4.5,...,20688388,UTIL,5400,SAS@LAC 12/20/2021 10:30PM ET,LAC,22.53,Eric Bledsoe,45.895956,18.368817,2.498580


In [41]:
result4.dtypes

first_name             object
last_name              object
position               object
position_alt           object
injury_status          object
game_date              object
slate                 float64
team                   object
opp                    object
spread                float64
over_under            float64
implied_team_score    float64
salary                  int64
L5_dvp_rank             int64
L5_ppg_floor          float64
L5_ppg_avg            float64
L5_ppg_max            float64
ppg_projection        float64
value_projection      float64
ppg_actual            float64
value_actual          float64
Name                   object
Position               object
Name + ID              object
ID                      int64
Roster Position        object
Salary                  int64
Game Info              object
TeamAbbrev             object
AvgPointsPerGame      float64
PlayerName             object
Average               float64
StandardDeviation     float64
Average_ST

In [43]:
selection = result4[['Name', 'AvgPointsPerGame', 'ppg_projection', 'Salary', 'Position', 'StandardDeviation']]

In [44]:
selection

,Name,AvgPointsPerGame,ppg_projection,Salary,Position,StandardDeviation
0,Paul George,46.28,43.5,12400,SG/SF,12.474737
2,Dejounte Murray,46.33,44.5,11400,PG,16.606912
3,Reggie Jackson,30.35,31.5,9200,PG,23.240824
4,Derrick White,30.39,31.0,8000,PG/SG,16.627761
5,Jakob Poeltl,30.62,32.2,7600,C,19.633243
6,Luke Kennard,22.07,24.6,7400,SG,28.104520
7,Keldon Johnson,28.01,26.5,7000,PF,20.052901
8,Terance Mann,21.95,23.3,6800,PG/SF,29.204982
9,Ivica Zubac,24.18,31.5,5800,C,18.835737
10,Eric Bledsoe,22.53,22.3,5400,PG,18.368817


In [45]:
Modified=selection

In [46]:
Modified.index = np.arange(1, len(Modified)+1)

In [47]:
Modified

,Name,AvgPointsPerGame,ppg_projection,Salary,Position,StandardDeviation
1,Paul George,46.28,43.5,12400,SG/SF,12.474737
2,Dejounte Murray,46.33,44.5,11400,PG,16.606912
3,Reggie Jackson,30.35,31.5,9200,PG,23.240824
4,Derrick White,30.39,31.0,8000,PG/SG,16.627761
5,Jakob Poeltl,30.62,32.2,7600,C,19.633243
6,Luke Kennard,22.07,24.6,7400,SG,28.104520
7,Keldon Johnson,28.01,26.5,7000,PF,20.052901
8,Terance Mann,21.95,23.3,6800,PG/SF,29.204982
9,Ivica Zubac,24.18,31.5,5800,C,18.835737
10,Eric Bledsoe,22.53,22.3,5400,PG,18.368817


In [48]:
Code_list=['x1x2','x3x4','x5x6','x7x8','x9x10',
           'x11x12','x13x14','x15x16','x17x18',
           'x19x20','x21x22','x23x24','x25x26',
           'x27x28','x29x30','x31x32','x33x34']

In [49]:
Modified.index = Code_list

In [51]:
Modified['season_average']=Modified['AvgPointsPerGame']

<ipython-input-51-9999f7edfade>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Modified['season_average']=Modified['AvgPointsPerGame']


In [52]:
Modified

,Name,AvgPointsPerGame,ppg_projection,Salary,Position,StandardDeviation,season_average
x1x2,Paul George,46.28,43.5,12400,SG/SF,12.474737,46.28
x3x4,Dejounte Murray,46.33,44.5,11400,PG,16.606912,46.33
x5x6,Reggie Jackson,30.35,31.5,9200,PG,23.240824,30.35
x7x8,Derrick White,30.39,31.0,8000,PG/SG,16.627761,30.39
x9x10,Jakob Poeltl,30.62,32.2,7600,C,19.633243,30.62
x11x12,Luke Kennard,22.07,24.6,7400,SG,28.104520,22.07
x13x14,Keldon Johnson,28.01,26.5,7000,PF,20.052901,28.01
x15x16,Terance Mann,21.95,23.3,6800,PG/SF,29.204982,21.95
x17x18,Ivica Zubac,24.18,31.5,5800,C,18.835737,24.18
x19x20,Eric Bledsoe,22.53,22.3,5400,PG,18.368817,22.53


In [54]:
Modified['AvgPointsPerGame'] = np.random.normal(Modified['ppg_projection'], Modified['StandardDeviation'])

<ipython-input-54-c2fc2681f826>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Modified['AvgPointsPerGame'] = np.random.normal(Modified['ppg_projection'], Modified['StandardDeviation'])


In [55]:
Modified

,Name,AvgPointsPerGame,ppg_projection,Salary,Position,StandardDeviation,season_average
x1x2,Paul George,41.286470,43.5,12400,SG/SF,12.474737,46.28
x3x4,Dejounte Murray,51.803907,44.5,11400,PG,16.606912,46.33
x5x6,Reggie Jackson,18.752470,31.5,9200,PG,23.240824,30.35
x7x8,Derrick White,36.689034,31.0,8000,PG/SG,16.627761,30.39
x9x10,Jakob Poeltl,44.269199,32.2,7600,C,19.633243,30.62
x11x12,Luke Kennard,28.594087,24.6,7400,SG,28.104520,22.07
x13x14,Keldon Johnson,47.923232,26.5,7000,PF,20.052901,28.01
x15x16,Terance Mann,-16.034323,23.3,6800,PG/SF,29.204982,21.95
x17x18,Ivica Zubac,50.565270,31.5,5800,C,18.835737,24.18
x19x20,Eric Bledsoe,-8.481082,22.3,5400,PG,18.368817,22.53


In [56]:
Download=Modified

In [57]:
m = GEKKO() # create GEKKO model
# create binary variables
x1 = m.Var(integer=True,lb=0,ub=1) 
x2 = m.Var(integer=True,lb=0,ub=1)
x3 = m.Var(integer=True,lb=0,ub=1)
x4 = m.Var(integer=True,lb=0,ub=1)
x5 = m.Var(integer=True,lb=0,ub=1)
x6 = m.Var(integer=True,lb=0,ub=1)
x7 = m.Var(integer=True,lb=0,ub=1)
x8 = m.Var(integer=True,lb=0,ub=1)
x9 = m.Var(integer=True,lb=0,ub=1)
x10 = m.Var(integer=True,lb=0,ub=1)
x11 = m.Var(integer=True,lb=0,ub=1)
x12 = m.Var(integer=True,lb=0,ub=1)
x13 = m.Var(integer=True,lb=0,ub=1)
x14 = m.Var(integer=True,lb=0,ub=1)
x15 = m.Var(integer=True,lb=0,ub=1)
x16 = m.Var(integer=True,lb=0,ub=1)
x17 = m.Var(integer=True,lb=0,ub=1)
x18 = m.Var(integer=True,lb=0,ub=1)
x19 = m.Var(integer=True,lb=0,ub=1)
x20 = m.Var(integer=True,lb=0,ub=1)
x21 = m.Var(integer=True,lb=0,ub=1)
x22 = m.Var(integer=True,lb=0,ub=1)
x23 = m.Var(integer=True,lb=0,ub=1)
x24 = m.Var(integer=True,lb=0,ub=1)
x25 = m.Var(integer=True,lb=0,ub=1)
x26 = m.Var(integer=True,lb=0,ub=1)
x27 = m.Var(integer=True,lb=0,ub=1)
x28 = m.Var(integer=True,lb=0,ub=1)
x29 = m.Var(integer=True,lb=0,ub=1) 
x30 = m.Var(integer=True,lb=0,ub=1)
x31 = m.Var(integer=True,lb=0,ub=1)
x32 = m.Var(integer=True,lb=0,ub=1)
x33 = m.Var(integer=True,lb=0,ub=1)
x34 = m.Var(integer=True,lb=0,ub=1)

#m.Equation(x2==1) 
#m.Equation(x4==1)
m.Equation(x6==1)

m.Equation(x2+x4+x6+x8+x10+x12+x14+x16+x18+x20+x22+x24+x26+x28+
           x30+x32+x34==1)

m.Equation(x1+x2+x3+x4+x5+x6+x7+x8+x9+x10+x11+x12+x13+x14+
           x15+x16+x17+x18+x19+x20+x21+x22+x23+x24+x25+x26+x27+x28+
           x29+x30+x31+x32+x33+x34== 6)

m.Equation(x1+x2<=1)
m.Equation(x3+x4<=1)
m.Equation(x5+x6<=1)
m.Equation(x7+x8<=1)
m.Equation(x9+x10<=1)
m.Equation(x11+x12<=1)
m.Equation(x13+x14<=1)
m.Equation(x15+x16<=1)
m.Equation(x17+x18<=1)
m.Equation(x19+x20<=1)
m.Equation(x21+x22<=1)
m.Equation(x23+x24<=1)
m.Equation(x25+x26<=1)
m.Equation(x27+x28<=1)
m.Equation(x29+x30<=1)
m.Equation(x31+x32<=1)
m.Equation(x33+x34<=1)

m.Equation((x1*Download.iloc[0]['Salary'])+(x2*Download.iloc[0]['Salary']*1.5)+
            (x3*Download.iloc[1]['Salary'])+(x4*Download.iloc[1]['Salary']*1.5)+
            (x5*Download.iloc[2]['Salary'])+(x6*Download.iloc[2]['Salary']*1.5)+
            (x7*Download.iloc[3]['Salary'])+(x8*Download.iloc[3]['Salary']*1.5)+
            (x9*Download.iloc[4]['Salary'])+(x10*Download.iloc[4]['Salary']*1.5)+
            (x11*Download.iloc[5]['Salary'])+(x12*Download.iloc[5]['Salary']*1.5)+
            (x13*Download.iloc[6]['Salary'])+(x14*Download.iloc[6]['Salary']*1.5)+
            (x15*Download.iloc[7]['Salary'])+(x16*Download.iloc[7]['Salary']*1.5)+
            (x17*Download.iloc[8]['Salary'])+(x18*Download.iloc[8]['Salary']*1.5)+
            (x19*Download.iloc[9]['Salary'])+(x20*Download.iloc[9]['Salary']*1.5)+
            (x21*Download.iloc[10]['Salary'])+(x22*Download.iloc[10]['Salary']*1.5)+
            (x23*Download.iloc[11]['Salary'])+(x24*Download.iloc[11]['Salary']*1.5)+
            (x25*Download.iloc[12]['Salary'])+(x26*Download.iloc[12]['Salary']*1.5)+
            (x27*Download.iloc[13]['Salary'])+(x28*Download.iloc[13]['Salary']*1.5)+
            (x29*Download.iloc[14]['Salary'])+(x30*Download.iloc[14]['Salary']*1.5)+
            (x31*Download.iloc[15]['Salary'])+(x32*Download.iloc[15]['Salary']*1.5)+
            (x33*Download.iloc[16]['Salary'])+(x34*Download.iloc[16]['Salary']*1.5)<=50000)


m.Maximize(((x1*Download.iloc[0]['AvgPointsPerGame'])+(x2*Download.iloc[0]['AvgPointsPerGame']*1.5)+
        (x3*Download.iloc[1]['AvgPointsPerGame'])+(x4*Download.iloc[1]['AvgPointsPerGame']*1.5)+
        (x5*Download.iloc[2]['AvgPointsPerGame'])+(x6*Download.iloc[2]['AvgPointsPerGame']*1.5)+
        (x7*Download.iloc[3]['AvgPointsPerGame'])+(x8*Download.iloc[3]['AvgPointsPerGame']*1.5)+
        (x9*Download.iloc[4]['AvgPointsPerGame'])+(x10*Download.iloc[4]['AvgPointsPerGame']*1.5)+
        (x11*Download.iloc[5]['AvgPointsPerGame'])+(x12*Download.iloc[5]['AvgPointsPerGame']*1.5)+
        (x13*Download.iloc[6]['AvgPointsPerGame'])+(x14*Download.iloc[6]['AvgPointsPerGame']*1.5)+
        (x15*Download.iloc[7]['AvgPointsPerGame'])+(x16*Download.iloc[7]['AvgPointsPerGame']*1.5)+
        (x17*Download.iloc[8]['AvgPointsPerGame'])+(x18*Download.iloc[8]['AvgPointsPerGame']*1.5)+
        (x19*Download.iloc[9]['AvgPointsPerGame'])+(x20*Download.iloc[9]['AvgPointsPerGame']*1.5)+
        (x21*Download.iloc[10]['AvgPointsPerGame'])+(x22*Download.iloc[10]['AvgPointsPerGame']*1.5)+
        (x23*Download.iloc[11]['AvgPointsPerGame'])+(x24*Download.iloc[11]['AvgPointsPerGame']*1.5)+
        (x25*Download.iloc[12]['AvgPointsPerGame'])+(x26*Download.iloc[12]['AvgPointsPerGame']*1.5)+
        (x27*Download.iloc[13]['AvgPointsPerGame'])+(x28*Download.iloc[13]['AvgPointsPerGame']*1.5)+
        (x29*Download.iloc[14]['AvgPointsPerGame'])+(x30*Download.iloc[14]['AvgPointsPerGame']*1.5)+
        (x31*Download.iloc[15]['AvgPointsPerGame'])+(x32*Download.iloc[15]['AvgPointsPerGame']*1.5)+
        (x33*Download.iloc[16]['AvgPointsPerGame'])+(x34*Download.iloc[16]['AvgPointsPerGame']*1.5)))

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')

d=[]
for i in range(17):
    



apm 108.183.51.97_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           52
   Intermediates:            0
   Connections  :            0
   Equations    :           22
   Residuals    :           22
 
 Number of state variables:             52
 Number of total equations: -           21
 Number of slack variables: -           18
 ---------------------------------------
 Degrees of freedom       :             13
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    0 Obj: -2.83E+02 Gap:  0.00E+00
 Successful solution
 
 ------------

In [61]:
#Download.iloc[16]['Name']
str(x34.value[0])

'0.0'

In [66]:
d=[]
for i in range(17):
    d.append(Download.iloc[i]['Name'])
    d.append(Download.iloc[i]['Name'])

In [67]:
d

['Paul George',
 'Paul George',
 'Dejounte Murray',
 'Dejounte Murray',
 'Reggie Jackson',
 'Reggie Jackson',
 'Derrick White',
 'Derrick White',
 'Jakob Poeltl',
 'Jakob Poeltl',
 'Luke Kennard',
 'Luke Kennard',
 'Keldon Johnson',
 'Keldon Johnson',
 'Terance Mann',
 'Terance Mann',
 'Ivica Zubac',
 'Ivica Zubac',
 'Eric Bledsoe',
 'Eric Bledsoe',
 'Nicolas Batum',
 'Nicolas Batum',
 'Devin Vassell',
 'Devin Vassell',
 'Justise Winslow',
 'Justise Winslow',
 'Bryn Forbes',
 'Bryn Forbes',
 'Keita Bates-Diop',
 'Keita Bates-Diop',
 'Tre Jones',
 'Tre Jones',
 'Amir Coffey',
 'Amir Coffey']